In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [5]:
df = pd.read_csv('../2. Dataset/fmnist_small.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,125,72,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,3,0,2,0,0,210,228,228,233,0,0,0,0,0,0,0,0,0,31,81,133,184,201,190,117,0,0,2,1,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0,43,117,34,15,24,33,117,80,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,51,144,152,...,157,158,161,148,159,58,0,6,0,0,0,0,0,0,0,0,0,4,0,60,143,143,148,146,152,152,148,148,147,145,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,0,0,0,2,0,33,114,37,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,11,17,13,13,11,10,7,5,5,5,7,0,0,0,1,0,0,41,69,88,86,94,106,114,118,47,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,0,2,0,58,145,114,10,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,...,165,153,155,134,143,172,215,62,0,0,0,0,0,0,0,0,10,190,178,194,209,211,209,205,211,215,213,217,225,228,213,203,174,151,188,10,0,0,0,0


In [6]:
x = df.iloc[:,1:].values / 255.0
y = df.iloc[:,0].values 

In [7]:
xtrain , xtest , ytrain , ytest = train_test_split(x,y,test_size=0.2,random_state=20)

In [8]:
xtrain_tensor = torch.from_numpy(xtrain).float()
xtest_tensor = torch.from_numpy(xtest).float()
ytrain_tensor = torch.from_numpy(ytrain)
ytest_tensor = torch.from_numpy(ytest)

In [10]:
class CustomDataset(Dataset):

    def __init__(self, features, labels):
        
        self.features = features
        self.labels = labels

    def __len__(self):
        
        return len(self.features)
    
    def __getitem__(self, idx):
        
        return self.features[idx], self.labels[idx]


In [13]:
train_dataset = CustomDataset(xtrain_tensor,ytrain_tensor)
test_dataset = CustomDataset(xtest_tensor,ytest_tensor)

In [14]:
class MyNN(nn.Module):

  def __init__(self,input_dim, output_dim, num_hidden_layers, neurons_per_layer, dropout_rate):

    super().__init__()

    layers = []

    for i in range(num_hidden_layers):

      layers.append(nn.Linear(input_dim, neurons_per_layer))
      layers.append(nn.BatchNorm1d(neurons_per_layer))
      layers.append(nn.ReLU())
      layers.append(nn.Dropout(dropout_rate))
      input_dim = neurons_per_layer
    
    layers.append(nn.Linear(neurons_per_layer, output_dim))

    self.network = nn.Sequential(*layers)

  def forward(self, features):

    out = self.network(features)

    return out

In [15]:
device = 'cpu'
if hasattr(torch,'mps') and torch.backends.mps.is_available():
    device = 'mps'
    print("MPS is available")

MPS is available


In [16]:
# objective function
def objective(trial):

  # Hyperparameter values from the search space
  
  num_hidden_layers = trial.suggest_int("num_hidden_layers", 1, 5)
  neurons_per_layer = trial.suggest_int("neurons_per_layer", 8, 128, step=8)
  dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1)

  epochs = trial.suggest_int("epochs", 10, 50, step=10)
  learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
  batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
  
  optimizer_name = trial.suggest_categorical("optimizer", ['Adam', 'SGD', 'RMSprop'])
  weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)

  # Dataloader

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

  # model init
  input_dim = 784
  output_dim = 10

  model = MyNN(input_dim, output_dim, num_hidden_layers, neurons_per_layer, dropout_rate)
  model.to(device)

  # optimizer selection
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)


  if optimizer_name == 'Adam':
      optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  elif optimizer_name == 'SGD':
      optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  else:
      optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)


  # Training loop

  for epoch in range(epochs):

    for batch_features, batch_labels in train_loader:

      # move data to gpu
      batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

      # forward pass
      outputs = model(batch_features)

      # calculate loss
      loss = criterion(outputs, batch_labels)

      # back pass
      optimizer.zero_grad()
      loss.backward()

      # update grads
      optimizer.step()


  # Evaluation 

  model.eval()

  total = 0
  correct = 0

  with torch.no_grad():

    for batch_features, batch_labels in test_loader:

      # move data to gpu
      batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

      outputs = model(batch_features)

      _, predicted = torch.max(outputs, 1)

      total = total + batch_labels.shape[0]

      correct = correct + (predicted == batch_labels).sum().item()

    accuracy = correct/total

  return accuracy

In [17]:
import optuna

study = optuna.create_study(direction='maximize')

/Users/akashjain/Desktop/Pytorch/Pyvenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-12-03 18:10:15,336] A new study created in memory with name: no-name-7cfcac2b-a98d-46be-a361-f91442f0bde9


In [18]:
study.optimize(objective, n_trials=10)

/Users/akashjain/Desktop/Pytorch/Pyvenv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
[I 2025-12-03 18:11:55,207] Trial 0 finished with value: 0.62 and parameters: {'num_hidden_layers': 4, 'neurons_per_layer': 8, 'dropout_rate': 0.2, 'epochs': 50, 'learning_rate': 0.003923593993445573, 'batch_size': 16, 'optimizer': 'SGD', 'weight_decay': 0.00034035167994113955}. Best is trial 0 with value: 0.62.
[I 2025-12-03 18:12:31,836] Trial 1 finished with value: 0.43833333333333335 and parameters: {'num_hidden_layers': 5, 'neurons_per_layer': 24, 'dropout_rate': 0.5, 'epochs': 30, 'learning_rate': 0.018089929301193675, 'batch_size': 16, 'optimizer': 'SGD', 'weight_decay': 0.0003716149924262154}. Best is trial 0 with value: 0.62.
[I 2025-12-03 18:13:20,824] Trial 2 finished with value: 0.6658333333333334 and parameters: {'num_hidden_layer

In [ ]:
study.best_value